In [1]:
import pandas as pd
import numpy as np


In [3]:
df = pd.read_csv('data/ALL-SAMPLES-quality_summary.tsv', sep='\t')
df.head()

,contig_id,contig_length,provirus,proviral_length,gene_count,viral_genes,host_genes,checkv_quality,miuvig_quality,completeness,completeness_method,contamination,kmer_freq,warnings
0,20100900_E1D_10_c_000000000306||0_partial,33678,No,NaN,27,2,7,Low-quality,Genome-fragment,15.58,HMM-based (lower-bound),0.0,1.0,NaN
1,20100900_E3D_12_c_000000003454||0_partial,11280,No,NaN,15,1,3,Low-quality,Genome-fragment,19.43,HMM-based (lower-bound),0.0,1.0,NaN
2,20110600_P1M_20_c_000000000591||0_partial,11850,No,NaN,23,2,1,Low-quality,Genome-fragment,21.15,HMM-based (lower-bound),0.0,1.0,NaN
3,20110600_P1M_22_c_000000003353||0_partial,23090,Yes,16433,26,4,5,Low-quality,Genome-fragment,22.48,HMM-based (lower-bound),28.83,1.0,NaN
4,20110600_P1M_3_c_000000000943||0_partial,21146,No,NaN,17,2,1,Low-quality,Genome-fragment,27.28,HMM-based (lower-bound),0.0,1.0,NaN


In [19]:
df = df.loc[df['contig_length'] != 'contig_length']
df['contig_length'] = df['contig_length'].astype(int)

<ipython-input-19-5e1335b38082>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['contig_length'] = df['contig_length'].astype(int)


In [20]:
df.checkv_quality.value_counts()

Not-determined    24015
Low-quality        7808
Medium-quality      347
Complete             23
High-quality         22
Name: checkv_quality, dtype: int64

In [22]:
# contig must be >10kb or checkv quality high or Complete
quality_contigs = df.loc[(df['contig_length'] > 10000) | ( (df['checkv_quality'] == 'High-quality') | (df['checkv_quality'] == 'Complete') )]

In [27]:
len(quality_contigs), len(df)

(7858, 32215)

In [26]:
# load vOTU representative headers
with open('data/vOTU_cluster_headers.txt', 'r') as f:
    headers = [i.lstrip('>').strip('\n') for i in f.readlines()]
headers[:10]

['20160700_E14.VIRSorter_NODE_1_length_351989_cov_12_429615_gene_124_gene_477-68188-352037-cat_5',
 'STM_0716_E_M_E034_E058_A.contigs_1kb_k121_584595||full',
 '20120700_P3D.VIRSorter_NODE_161_length_217151_cov_11_826464-circular-cat_2',
 '20130700_S2X.VIRSorter_NODE_3_length_197518_cov_59_067982-cat_2',
 '20140700_S11.VIRSorter_NODE_5_length_190782_cov_9_251422-cat_2',
 '20110700_S1S.VIRSorter_NODE_1_length_186748_cov_10_597912-cat_2',
 '20120800_E2X.VIRSorter_NODE_52_length_167441_cov_24_979359-circular-cat_2',
 '20160700_S15.VIRSorter_NODE_2_length_166797_cov_9_507389-cat_2',
 '20120800_E1X.VIRSorter_NODE_27_length_162874_cov_6_971084-cat_2',
 'Bog.Labelled.R2.T2.F1.trimmomatic.metaspades_bin.69_84.77_5.37_NODE_16_length_161907_cov_18.638831||full']

In [30]:
filtered_headers = [i for i in headers if i in list(quality_contigs.contig_id)]
len(filtered_headers)

1849

In [31]:
filtered_headers

['STM_0716_E_M_E034_E058_A.contigs_1kb_k121_584595||full',
 'Bog.Labelled.R2.T2.F1.trimmomatic.metaspades_bin.69_84.77_5.37_NODE_16_length_161907_cov_18.638831||full',
 'STM_0716_E_M_E058_A.contigs_1kb_k121_840231||full',
 '3300037141_6_Ga0395915_0000074||full',
 'STM_0716_E_M_E034_E058_E069_D.contigs_1kb_k121_960899||full',
 'PLDW01_ENA|PLDW01000091|PLDW01000091.1||full',
 'STM_0716_E_M_E026_E030_E034_A.contigs_1kb_k121_2486055||full',
 'PMDZ01_ENA|PMDZ01000006|PMDZ01000006.1||full',
 'STM_0716_E_M_E026_A.contigs_1kb_k121_1111751||full',
 'Palsa.Unlabelled.R1.T3.F1.JGI.BFC.metaspades_bin.69_93.69_1.67_scaffold_210_c1||full',
 'Bog.None.R1.T0.trimmomatic.megahit_bin.316_95.55_4.21_k141_3266318||full',
 'STM_0716_E_M_E034_E058_E069_D.contigs_1kb_k121_4428572||full',
 'PMOH01_ENA|PMOH01000316|PMOH01000316.1||full',
 'STM_0716_E_M_E026_E050_A.contigs_1kb_k121_661925||full',
 'Bog.Labelled.R1.T2.F3.trimmomatic.metaspades_bin.118_92.33_1.72_NODE_214_length_73431_cov_5.745871||full',
 '20170

# check to make sure all complete circular are in the cluster list

In [36]:
with open('checkv_10kb_or_high_quality_vOTUs.txt', 'w') as f:
    for i in filtered_headers:
        f.write('>' + i + '\n')

# check for prophage

In [38]:
df['provirus'].value_counts()

No     30650
Yes     1565
Name: provirus, dtype: int64

In [39]:
quality_contigs['provirus'].value_counts()

No     7074
Yes     784
Name: provirus, dtype: int64